# **Applying Advanced Transformations**

_John Andrew Dixon_

---

##### **Imports**

In [291]:
import json
import numpy as np
import pandas as pd

##### **Data Load**

In [292]:
# Make it so that all of the dataframe's columns are shown
pd.set_option('display.max_columns', None)

In [293]:
# Remote URL to the superhero powers data
powers_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQ2cMaGI74T_DqeCWDojRIyUCQqNZxsu_L3f42JJKV-_f873im-CBttJU8qn_Kan22qa71uCXfDWdMI/pub?output=csv"
# Load the data
powers_df = pd.read_csv(powers_url)
# Verify it loaded
powers_df.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [294]:
# Remote URL to the superhero info data
info_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRgXxC_AvDsxnrZSMIuKnPMvrSrL6ZnoLogMNU-ZD8uOB88pbsINNtkii1PlMTgWPSyrqm5JJnU6pCr/pub?output=csv"
# Load the data
info_df = pd.read_csv(info_url)
# Verify it loaded
info_df.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


---

## **Cleaning**

##### **_Check For Missing Values_**

In [295]:
# Check for missing values
info_df.isna().sum().sum()

0

In [296]:
# Check for missing values
powers_df.isna().sum().sum()

0

##### **_Check For Duplicates_**

In [297]:
# Check for duplicates
info_df.duplicated().sum()

0

In [298]:
# Check for duplicates
powers_df.duplicated().sum()

0

##### **_Separate Hero From Publisher_**

In [299]:
# Split the "Hero|Publisher" column using split and save the 
# result to two new columns named "Hero" and "Publisher"
info_df[["Hero", "Publisher"]] = info_df["Hero|Publisher"].str.split("|", expand=True)
# Drop the original column
info_df.drop("Hero|Publisher", axis=1, inplace=True)
# Verify it worked
info_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


##### **_Separate Height From Weight_**

In [300]:
a = info_df.loc[0, "Measurements"]
type(json.loads(a.replace("'", '"')))

dict

In [301]:
# Replace all single quotes in the strings on the 
# "Measurements" column
info_df["Measurements"] = info_df["Measurements"].str.replace("'", '"')
# Apply JSON loads to the whole column to make the 
# string into a dictionary
info_df["Measurements"] = info_df["Measurements"].apply(json.loads)
# Verify it worked
info_df["Measurements"].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [302]:
# Create a DataFrame with just the height and weights
heights_weights = info_df["Measurements"].apply(pd.Series)
heights_weights.head()

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg


In [303]:
# Rename the columns of the height weight dataframe
heights_weights.rename(columns = {"Height": "Height (cm)", "Weight": "Weight (kg)"}, inplace=True)
heights_weights.head()

,Height (cm),Weight (kg)
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg


In [304]:
# Remove the units from the measurements strings and then cast each as a float
heights_weights["Height (cm)"] = heights_weights["Height (cm)"].str.split(" ", expand=True)[0].astype(float)
heights_weights["Weight (kg)"] = heights_weights["Weight (kg)"].str.split(" ", expand=True)[0].astype(float)
# Verify they are now floats
heights_weights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Height (cm)  463 non-null    float64
 1   Weight (kg)  463 non-null    float64
dtypes: float64(2)
memory usage: 7.4 KB


In [305]:
# Add the heights_weights dataframe to the original
info_df = pd.concat([info_df, heights_weights], axis=1)
info_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height (cm),Weight (kg)
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics,193.0,122.0


In [306]:
# Remove the original "Measurements" column
info_df.drop(columns="Measurements", inplace=True)
info_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height (cm),Weight (kg)
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


##### **_Rearrange Info Columns_**

> _**Note:** I am doing this because I don't like the current arrangement of the `info_df` columns. I want them to somewhat match the arrangement in the learn plaform but with a slight variation that makes sense. Please don't make me resubmit because I rearranged columns._

In [307]:
# Make the columns into a list
columns = list(info_df.columns)
columns

['Gender',
 'Race',
 'Alignment',
 'Hair color',
 'Eye color',
 'Skin color',
 'Hero',
 'Publisher',
 'Height (cm)',
 'Weight (kg)']

In [308]:
# Rearrange columns
columns = [columns[columns.index("Hero")], \
           columns[columns.index("Publisher")], \
           columns[columns.index("Alignment")],\
           columns[columns.index("Gender")], \
           columns[columns.index("Race")], \
           columns[columns.index("Height (cm)")], \
           columns[columns.index("Weight (kg)")], \
           columns[columns.index("Skin color")], \
           columns[columns.index("Eye color")], \
           columns[columns.index("Hair color")]]
columns

['Hero',
 'Publisher',
 'Alignment',
 'Gender',
 'Race',
 'Height (cm)',
 'Weight (kg)',
 'Skin color',
 'Eye color',
 'Hair color']

In [309]:
# Apply the new rearrangement to the old info_df
info_df = info_df[columns]
info_df.head()

,Hero,Publisher,Alignment,Gender,Race,Height (cm),Weight (kg),Skin color,Eye color,Hair color
0,A-Bomb,Marvel Comics,good,Male,Human,203.0,441.0,Unknown,yellow,No Hair
1,Abe Sapien,Dark Horse Comics,good,Male,Icthyo Sapien,191.0,65.0,blue,blue,No Hair
2,Abin Sur,DC Comics,good,Male,Ungaran,185.0,90.0,red,blue,No Hair
3,Abomination,Marvel Comics,bad,Male,Human / Radiation,203.0,441.0,Unknown,green,No Hair
4,Absorbing Man,Marvel Comics,bad,Male,Human,193.0,122.0,Unknown,blue,No Hair


##### **_One-hot Encode Powers_**

In [310]:
# Split the "Powers" string by commas into a list
powers_df["Powers Split"] = powers_df["Powers"].str.split(",")
powers_df.head()

,hero_names,Powers,Powers Split
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...","[Agility, Accelerated Healing, Cold Resistance..."
3,Abin Sur,Lantern Power Ring,[Lantern Power Ring]
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...","[Accelerated Healing, Intelligence, Super Stre..."


> _**Note:** I alphabetized the powers for easier viewing. Please don't make me resubmit because I did this._

In [311]:
# Create a new DataFrame of exploded powers
powers_exploded = powers_df.explode("Powers Split")
# Find the unique powers in the exploded column
unique_powers = np.sort(powers_exploded["Powers Split"].unique())
unique_powers

array(['Accelerated Healing', 'Adaptation', 'Agility',
       'Animal Attributes', 'Animal Control', 'Animal Oriented Powers',
       'Animation', 'Anti-Gravity', 'Astral Projection', 'Astral Travel',
       'Audio Control', 'Banish', 'Biokinesis', 'Camouflage',
       'Changing Armor', 'Clairvoyance', 'Cloaking', 'Cold Resistance',
       'Cryokinesis', 'Danger Sense', 'Darkforce Manipulation',
       'Death Touch', 'Density Control', 'Dexterity',
       'Dimensional Awareness', 'Dimensional Travel', 'Duplication',
       'Durability', 'Echolocation', 'Elasticity', 'Electrical Transport',
       'Electrokinesis', 'Element Control',
       'Elemental Transmogrification', 'Empathy', 'Energy Absorption',
       'Energy Armor', 'Energy Beams', 'Energy Blasts',
       'Energy Constructs', 'Energy Manipulation', 'Energy Resistance',
       'Enhanced Hearing', 'Enhanced Memory', 'Enhanced Senses',
       'Enhanced Sight', 'Enhanced Smell', 'Enhanced Touch',
       'Fire Control', 'Fire Resis

In [312]:
# Create each power column and add true/false values if a superhero has 
# the following power. There was a warning here when I did it the way that
# the learn platform has so I did it another way instead that gives the same
# results
for power in unique_powers:
    power_df = pd.DataFrame({power: powers_df["Powers"].str.contains(power)})
    powers_df = pd.concat([powers_df, power_df], axis=1)
powers_df.head()

,hero_names,Powers,Powers Split,Accelerated Healing,Adaptation,Agility,Animal Attributes,Animal Control,Animal Oriented Powers,Animation,Anti-Gravity,Astral Projection,Astral Travel,Audio Control,Banish,Biokinesis,Camouflage,Changing Armor,Clairvoyance,Cloaking,Cold Resistance,Cryokinesis,Danger Sense,Darkforce Manipulation,Death Touch,Density Control,Dexterity,Dimensional Awareness,Dimensional Travel,Duplication,Durability,Echolocation,Elasticity,Electrical Transport,Electrokinesis,Element Control,Elemental Transmogrification,Empathy,Energy Absorption,Energy Armor,Energy Beams,Energy Blasts,Energy Constructs,Energy Manipulation,Energy Resistance,Enhanced Hearing,Enhanced Memory,Enhanced Senses,Enhanced Sight,Enhanced Smell,Enhanced Touch,Fire Control,Fire Resistance,Flight,Force Fields,Gliding,Gravity Control,Grim Reaping,Hair Manipulation,Heat Generation,Heat Resistance,Hyperkinesis,Hypnokinesis,Illumination,Illusions,Immortality,Insanity,Intangibility,Intelligence,Intuitive aptitude,Invisibility,Invulnerability,Jump,Lantern Power Ring,Levitation,Light Control,Longevity,Magic,Magic Resistance,Magnetism,Marksmanship,Matter Absorption,Melting,Mind Blast,Mind Control,Mind Control Resistance,Molecular Combustion,Molecular Dissipation,Molecular Manipulation,Natural Armor,Natural Weapons,Nova Force,Odin Force,Omnilingualism,Omnipotent,Omnipresent,Omniscient,Omnitrix,Peak Human Condition,Phasing,Phoenix Force,Photographic Reflexes,Plant Control,Portal Creation,Possession,Power Absorption,Power Augmentation,Power Cosmic,Power Nullifier,Power Sense,Power Suit,Precognition,Probability Manipulation,Projection,Psionic Powers,Qwardian Power Ring,Radar Sense,Radiation Absorption,Radiation Control,Radiation Immunity,Reality Warping,Reflexes,Regeneration,Resurrection,Seismic Power,Self-Sustenance,Shapeshifting,Size Changing,Sonar,Sonic Scream,Spatial Awareness,Speed Force,Stamina,Stealth,Sub-Mariner,Substance Secretion,Summoning,Super Breath,Super Speed,Super Strength,Symbiote Costume,Technopath/Cyberpath,Telekinesis,Telepathy,Telepathy Resistance,Teleportation,Terrakinesis,The Force,Thirstokinesis,Time Manipulation,Time Travel,Toxin and Disease Control,Toxin and Disease Resistance,Underwater breathing,Vision - Cryo,Vision - Heat,Vision - Infrared,Vision - Microscopic,Vision - Night,Vision - Telescopic,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed","[Agility, Super Strength, Stamina, Super Speed]",False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...","[Accelerated Healing, Durability, Longevity, S...",True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [313]:
# Drop the now unnecessary Power and Power Split columns
powers_df.drop(columns=["Powers", "Powers Split"], inplace=True)
powers_df.head()

,hero_names,Accelerated Healing,Adaptation,Agility,Animal Attributes,Animal Control,Animal Oriented Powers,Animation,Anti-Gravity,Astral Projection,Astral Travel,Audio Control,Banish,Biokinesis,Camouflage,Changing Armor,Clairvoyance,Cloaking,Cold Resistance,Cryokinesis,Danger Sense,Darkforce Manipulation,Death Touch,Density Control,Dexterity,Dimensional Awareness,Dimensional Travel,Duplication,Durability,Echolocation,Elasticity,Electrical Transport,Electrokinesis,Element Control,Elemental Transmogrification,Empathy,Energy Absorption,Energy Armor,Energy Beams,Energy Blasts,Energy Constructs,Energy Manipulation,Energy Resistance,Enhanced Hearing,Enhanced Memory,Enhanced Senses,Enhanced Sight,Enhanced Smell,Enhanced Touch,Fire Control,Fire Resistance,Flight,Force Fields,Gliding,Gravity Control,Grim Reaping,Hair Manipulation,Heat Generation,Heat Resistance,Hyperkinesis,Hypnokinesis,Illumination,Illusions,Immortality,Insanity,Intangibility,Intelligence,Intuitive aptitude,Invisibility,Invulnerability,Jump,Lantern Power Ring,Levitation,Light Control,Longevity,Magic,Magic Resistance,Magnetism,Marksmanship,Matter Absorption,Melting,Mind Blast,Mind Control,Mind Control Resistance,Molecular Combustion,Molecular Dissipation,Molecular Manipulation,Natural Armor,Natural Weapons,Nova Force,Odin Force,Omnilingualism,Omnipotent,Omnipresent,Omniscient,Omnitrix,Peak Human Condition,Phasing,Phoenix Force,Photographic Reflexes,Plant Control,Portal Creation,Possession,Power Absorption,Power Augmentation,Power Cosmic,Power Nullifier,Power Sense,Power Suit,Precognition,Probability Manipulation,Projection,Psionic Powers,Qwardian Power Ring,Radar Sense,Radiation Absorption,Radiation Control,Radiation Immunity,Reality Warping,Reflexes,Regeneration,Resurrection,Seismic Power,Self-Sustenance,Shapeshifting,Size Changing,Sonar,Sonic Scream,Spatial Awareness,Speed Force,Stamina,Stealth,Sub-Mariner,Substance Secretion,Summoning,Super Breath,Super Speed,Super Strength,Symbiote Costume,Technopath/Cyberpath,Telekinesis,Telepathy,Telepathy Resistance,Teleportation,Terrakinesis,The Force,Thirstokinesis,Time Manipulation,Time Travel,Toxin and Disease Control,Toxin and Disease Resistance,Underwater breathing,Vision - Cryo,Vision - Heat,Vision - Infrared,Vision - Microscopic,Vision - Night,Vision - Telescopic,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,3-D Man,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

##### _Merge Tables For Final DataFrame_

In [314]:
# Merge the tables for values where "Hero" = "hero_names" and then drop the "hero_names" column
final_df = pd.merge(info_df, powers_df, left_on="Hero", right_on="hero_names").drop(columns="hero_names")
final_df

,Hero,Publisher,Alignment,Gender,Race,Height (cm),Weight (kg),Skin color,Eye color,Hair color,Accelerated Healing,Adaptation,Agility,Animal Attributes,Animal Control,Animal Oriented Powers,Animation,Anti-Gravity,Astral Projection,Astral Travel,Audio Control,Banish,Biokinesis,Camouflage,Changing Armor,Clairvoyance,Cloaking,Cold Resistance,Cryokinesis,Danger Sense,Darkforce Manipulation,Death Touch,Density Control,Dexterity,Dimensional Awareness,Dimensional Travel,Duplication,Durability,Echolocation,Elasticity,Electrical Transport,Electrokinesis,Element Control,Elemental Transmogrification,Empathy,Energy Absorption,Energy Armor,Energy Beams,Energy Blasts,Energy Constructs,Energy Manipulation,Energy Resistance,Enhanced Hearing,Enhanced Memory,Enhanced Senses,Enhanced Sight,Enhanced Smell,Enhanced Touch,Fire Control,Fire Resistance,Flight,Force Fields,Gliding,Gravity Control,Grim Reaping,Hair Manipulation,Heat Generation,Heat Resistance,Hyperkinesis,Hypnokinesis,Illumination,Illusions,Immortality,Insanity,Intangibility,Intelligence,Intuitive aptitude,Invisibility,Invulnerability,Jump,Lantern Power Ring,Levitation,Light Control,Longevity,Magic,Magic Resistance,Magnetism,Marksmanship,Matter Absorption,Melting,Mind Blast,Mind Control,Mind Control Resistance,Molecular Combustion,Molecular Dissipation,Molecular Manipulation,Natural Armor,Natural Weapons,Nova Force,Odin Force,Omnilingualism,Omnipotent,Omnipresent,Omniscient,Omnitrix,Peak Human Condition,Phasing,Phoenix Force,Photographic Reflexes,Plant Control,Portal Creation,Possession,Power Absorption,Power Augmentation,Power Cosmic,Power Nullifier,Power Sense,Power Suit,Precognition,Probability Manipulation,Projection,Psionic Powers,Qwardian Power Ring,Radar Sense,Radiation Absorption,Radiation Control,Radiation Immunity,Reality Warping,Reflexes,Regeneration,Resurrection,Seismic Power,Self-Sustenance,Shapeshifting,Size Changing,Sonar,Sonic Scream,Spatial Awareness,Speed Force,Stamina,Stealth,Sub-Mariner,Substance Secretion,Summoning,Super Breath,Super Speed,Super Strength,Symbiote Costume,Technopath/Cyberpath,Telekinesis,Telepathy,Telepathy Resistance,Teleportation,Terrakinesis,The Force,Thirstokinesis,Time Manipulation,Time Travel,Toxin and Disease Control,Toxin and Disease Resistance,Underwater breathing,Vision - Cryo,Vision - Heat,Vision - Infrared,Vision - Microscopic,Vision - Night,Vision - Telescopic,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,A-Bomb,Marvel Comics,good,Male,Human,203.0,441.0,Unknown,yellow,No Hair,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Abe Sapien,Dark Horse Comics,good,Male,Icthyo Sapien,191.0,65.0,blue,blue,No Hair,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,

## **Questions**

**_Compare the average weight of super heroes who have Super Speed to those who do not._**


In [315]:
# Group the dataframe by super speed and find the mean of the weight
final_df.groupby("Super Speed")[["Weight (kg)"]].mean()

,Weight (kg)
Super Speed,
False,101.773585
True,129.404040


The average weight of those who have super speed is much higher than those who do not. This is funny. A Super Hero who is likelier to have supepr speed is likelier to be heavier. One would expect the opposite.

**_What is the average height of heroes for each publisher?_**

In [316]:
# Group the dataframe by "Publisher" and find the average "Height (cm)" of the heroes
# for each publisher
final_df.groupby("Publisher")[["Height (cm)"]].mean().sort_values(by="Height (cm)")

,Height (cm)
Publisher,
George Lucas,159.600000
Shueisha,171.500000
Dark Horse Comics,176.909091
Unknown,178.000000
Team Epic TV,180.750000
Star Trek,181.500000
DC Comics,181.923913
Marvel Comics,191.546128
Image Comics,211.000000
